# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.365138     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.477531     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
model_all.coefficients.print_rows(num_rows=20)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.05595049576 |  None  |
|     bedrooms     |  None |        0.0         |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None | 8468.531086910105  |  None  |
|   sqft_living    |  None | 24.42072098244546  |  None  |
| sqft_living_sqrt |  None | 350.06055338605546 |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None |        0.0         |  None  |
|  floors_square   |  None |        0.0         |  None  |
|    waterfront    |  None |        0.0         |  None  |
|       view       |  None |        0.0         |  None  |
|    condition     |  None |        0.0         |  None  |
|      grade       |  None | 842.0680348976231  |  None 

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

    - bathrooms
    - sqft_living
    - sqft_living_sqrt
    - grade
    - sqft_above
    

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [8]:
import numpy as np
rss = []
l1_p_array = []
for l1_penalty in np.logspace(1, 7, num=13):
    model = turicreate.linear_regression.create(training,
                                                target='price',
                                                features=all_features,
                                                l1_penalty=l1_penalty,
                                                l2_penalty=0,
                                                validation_set=None,
                                                verbose=False)
    predictions = model.predict(validation)
    residuals = predictions - validation['price']
    residuals *= residuals
    rss.append(residuals.sum())
    l1_p_array.append(l1_penalty)
    print('{:.2e}'.format(l1_penalty), '\t', '{:.10e}'.format(residuals.sum()))

1.00e+01 	 6.2576628514e+14
3.16e+01 	 6.2576628536e+14
1.00e+02 	 6.2576628606e+14
3.16e+02 	 6.2576628826e+14
1.00e+03 	 6.2576629521e+14
3.16e+03 	 6.2576631721e+14
1.00e+04 	 6.2576638676e+14
3.16e+04 	 6.2576660675e+14
1.00e+05 	 6.2576730279e+14
3.16e+05 	 6.2576950764e+14
1.00e+06 	 6.2577651773e+14
3.16e+06 	 6.2579906285e+14
1.00e+07 	 6.2588371909e+14


***QUIZ QUESTION.*** What was the best value for the `l1_penalty`?

    10

In [9]:
model_10 = turicreate.linear_regression.create(training,
                                               target='price',
                                               features=all_features,
                                               validation_set=None,
                                               l1_penalty=10,
                                               l2_penalty=0)

print(model_10.coefficients.print_rows(num_rows = 20))
print(model_10.coefficients['value'].nnz())

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 10        | 11       | 0.000003  | 0.583937     | 5329815.359960     | 273608.586502                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |   18993.4272127706   |  None  |
|     bedrooms     |  None |  7936.9676790313015  |  None  |
| bedrooms_square  |  None |  936.9933681932994   |  None  |
|    bathrooms     |  None |  25409.588934120668  |  None  |
|   sqft_living    |  None |  39.11513637970764   |  None  |
| sqft_living_sqrt |  None |  1124.650212807717   |  None  |
|     sqft_lot     |  None | 0.003483618222989654 |  None  |
|  sqft_lot_sqrt   |  None |  148.25839101140826  |  None  |
|      floors      |  None |  21204.335466950117  |  None  |
|  floors_square   |  None |  12915.524336072433  |  None  |
|    waterfront    |  None |   601905.594545272   |  None  |
|       view       |  None |  93312.85731187189   |  None  |
|    condition     |  None |  6609.035712447216   |  None  |
|      grade       |  No

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

    18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [10]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [11]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [12]:
nnz_array = []
for l1_penalty in l1_penalty_values:
    model = turicreate.linear_regression.create(training,
                                                target='price',
                                                features=all_features,
                                                l1_penalty=l1_penalty,
                                                l2_penalty=0,
                                                validation_set=None,
                                                verbose=False)
    nnz_value = model.coefficients['value'].nnz()
    nnz_array.append(nnz_value)
    #print('nnz: ', nnz_value,'max: ', '{:.2e}'.format(l1_penalty))
    print('nnz: ', nnz_value,'max: ', l1_penalty)
        

nnz:  18 max:  100000000.0
nnz:  18 max:  127427498.57031322
nnz:  18 max:  162377673.91887242
nnz:  18 max:  206913808.111479
nnz:  17 max:  263665089.87303555
nnz:  17 max:  335981828.6283788
nnz:  17 max:  428133239.8719396
nnz:  17 max:  545559478.1168514
nnz:  17 max:  695192796.1775591
nnz:  16 max:  885866790.4100832
nnz:  15 max:  1128837891.6846883
nnz:  15 max:  1438449888.2876658
nnz:  13 max:  1832980710.8324375
nnz:  12 max:  2335721469.0901213
nnz:  10 max:  2976351441.6313133
nnz:  6 max:  3792690190.7322536
nnz:  5 max:  4832930238.571753
nnz:  3 max:  6158482110.6602545
nnz:  1 max:  7847599703.514623
nnz:  1 max:  10000000000.0


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [13]:
l1_penalty_min = 2.97e+09
l1_penalty_max = 3.79e+09

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

    min: 2.97e+09
    max: 3.79e+09 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [14]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [15]:
rss = []
l1_p_array = []
for l1_penalty in l1_penalty_values:
    model = turicreate.linear_regression.create(training,
                                                target='price',
                                                features=all_features,
                                                l1_penalty=l1_penalty,
                                                l2_penalty=0,
                                                validation_set=None,
                                                verbose=False)
    predictions = model.predict(validation)
    residuals = predictions - validation['price']
    residuals *= residuals
    rss.append(residuals.sum())
    l1_p_array.append(l1_penalty)
    #print('{:.2e}'.format(l1_penalty), '\t', '{:.10e}'.format(residuals.sum()))
    print(l1_penalty, '\t', '{:.10e}'.format(residuals.sum()))    
    print(model.coefficients.print_rows(num_rows=20))

2970000000.0 	 9.6588341965e+14
+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 195736.93367659138 |  None  |
|     bedrooms     |  None | 2198.5122491853613 |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None | 17985.213991799552 |  None  |
|   sqft_living    |  None | 34.16034786792188  |  None  |
| sqft_living_sqrt |  None | 790.3749994815937  |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None | 3711.4219505604297 |  None  |
|  floors_square   |  None |        0.0         |  None  |
|    waterfront    |  None |        0.0         |  None  |
|       view       |  None | 11549.164785608646 |  None  |
|    condition     |  None |        0.0         |  None  |
|      grade       |  No

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?

    3444736842

2. What features in this model have non-zero coefficients?

    - bedrooms
    - bathrooms
    - sqft_living
    - sqft_living_sqrt
    - grade
    - sqft_above